In [ ]:
import numpy as np
import json
import slowquant.SlowQuant as sq
from slowquant.unitary_coupled_cluster.ucc_wavefunction import WaveFunctionUCC

import pyscf
from pyscf import mcscf, ao2mo

geometries = ["""Li 0.0 0.0 -0.65; H 0.0 0.0 0.65;""", 
             """Li 0.0 0.0 -0.70; H 0.0 0.0 0.70;""",
             """Li 0.0 0.0 -0.75; H 0.0 0.0 0.75;""",
             """Li 0.0 0.0 -0.80; H 0.0 0.0 0.80;""",
             """Li 0.0 0.0 -0.85; H 0.0 0.0 0.85;""",
             """Li 0.0 0.0 -0.90; H 0.0 0.0 0.90;""",
             """Li 0.0 0.0 -0.95; H 0.0 0.0 0.95;""",
             """Li 0.0 0.0 -1.00; H 0.0 0.0 1.00;""",
             """Li 0.0 0.0 -1.25; H 0.0 0.0 1.25;""",
             """Li 0.0 0.0 -1.50; H 0.0 0.0 1.50;""",
             """Li 0.0 0.0 -1.75; H 0.0 0.0 1.75;""",
             """Li 0.0 0.0 -2.00; H 0.0 0.0 2.00;""",
             """Li 0.0 0.0 -2.25; H 0.0 0.0 2.25;""",
             """Li 0.0 0.0 -2.50; H 0.0 0.0 2.50;""",
             """Li 0.0 0.0 -3.00; H 0.0 0.0 3.00;""",
             """Li 0.0 0.0 -3.50; H 0.0 0.0 3.50;""",]#Angstrom

results_true = []
results_false = []

basis = "cc-pvdz"
cas = (4, 10) # active space
unit = "angstrom"  #angstrom, bohr

for i, geometry in enumerate(geometries):
    print(f"\nProcessing geometry {i+1}:")
    print(geometry)
    #### SlowQuant
    SQobj = sq.SlowQuant()
    SQobj.set_molecule(geometry, distance_unit=unit,)
    SQobj.set_basis_set(basis)

    #### PySCF
    mol = pyscf.M(atom = geometry, basis = basis, unit = unit)
    mf = mol.RHF().run()

    #### HF 
    mo_coeffs = mf.mo_coeff

    #### get the integrals in the AO basis
    hcore_ao = mol.intor_symmetric('int1e_kin') + mol.intor_symmetric('int1e_nuc')
    eri_4fold_ao = mol.intor('int2e_sph')

    # OO-UCCSD
    WF = WaveFunctionUCC(
    num_elec=SQobj.molecule.number_electrons,
    cas=cas,
    mo_coeffs=mo_coeffs,
    h_ao=hcore_ao,
    g_ao=eri_4fold_ao,
    excitations="SD",
    include_active_kappa=True,
    )
    
    WF_false = WF
    WF_true = WF
    
    #WF_true.run_wf_optimization_1step(
   	#	 optimizer_name="SLSQP", 
   	#	 tol = 1e-4,
   	#	 orbital_optimization=True)
    
    WF_false.run_wf_optimization_1step(
    		optimizer_name="SLSQP", 
    		tol = 1e-4,
    		orbital_optimization=False)
   
    #energy_true = WF_true.energy_elec + mol.energy_nuc()
    energy_false = WF_false.energy_elec + mol.energy_nuc()

    #print(f"oo-UCCSD energy for geometry {i+1} = {energy_true}")
    
    print(f"UCCSD energy for geometry {i+1} = {energy_false}")

    
    #results_true.append({
     #   'geometry': geometry,
      #  'energy': energy_true,
       # 'nuclear_repulsion': mol.energy_nuc(),
        #'electronic_energy': WF_true.energy_elec
    #})
    
    results_false.append({
        'geometry': geometry,
        'energy': energy_false,
        'nuclear_repulsion': mol.energy_nuc(),
        'electronic_energy': WF_false.energy_elec
    })

    with open('results_false_pvdz_LiH_4.json', 'w') as f:
        json.dump(results_false, f, indent=4)  
    
    with open('results_true_pvdz_LiH_4.json', 'w') as f:
        json.dump(results_true, f, indent=4)


Processing geometry 1:
Li 0.0 0.0 -0.65; H 0.0 0.0 0.65;
converged SCF energy = -7.96601167739468
### Parameters information:
### Number kappa: 0
### Number theta1: 16
### Number theta2: 136
### Number theta3: 0
### Number theta4: 0
### Number theta5: 0
### Number theta6: 0
### Total parameters: 152

Iteration # | Iteration time [s] | Electronic energy [Hartree]
--------     1      |        27.23       |     -9.1795986731041879    


In [2]:
import json

### CC-PVDZ
with open('results_true_pvdz_LiH.json', 'r') as f:
    loaded_results_true_pvdz_LiH = json.load(f)

with open('results_false_pvdz_LiH.json', 'r') as f:
    loaded_results_false_pvdz_LiH = json.load(f)

print(f'Results for CC-PVDZ')
print()
print(loaded_results_true_pvdz_LiH)
print()
print(loaded_results_false_pvdz_LiH) 

Results for CC-PVDZ

[{'geometry': 'Li 0.0 0.0 -0.65; H 0.0 0.0 0.65;', 'energy': -7.9966960016359305, 'nuclear_repulsion': 1.221178179046154, 'electronic_energy': -9.217874180682085}, {'geometry': 'Li 0.0 0.0 -0.70; H 0.0 0.0 0.70;', 'energy': -8.00648675364148, 'nuclear_repulsion': 1.133951166257143, 'electronic_energy': -9.140437919898623}, {'geometry': 'Li 0.0 0.0 -0.75; H 0.0 0.0 0.75;', 'energy': -8.01156247313749, 'nuclear_repulsion': 1.05835442184, 'electronic_energy': -9.06991689497749}, {'geometry': 'Li 0.0 0.0 -0.80; H 0.0 0.0 0.80;', 'energy': -8.013287033473329, 'nuclear_repulsion': 0.992207270475, 'electronic_energy': -9.005494303948328}, {'geometry': 'Li 0.0 0.0 -0.85; H 0.0 0.0 0.85;', 'energy': -8.0126267189904, 'nuclear_repulsion': 0.9338421369176472, 'electronic_energy': -8.946468855908048}, {'geometry': 'Li 0.0 0.0 -0.90; H 0.0 0.0 0.90;', 'energy': -8.010276035193062, 'nuclear_repulsion': 0.8819620182, 'electronic_energy': -8.892238053393061}, {'geometry': 'Li 0.0 

In [ ]:
import os
from qiskit_nwchem_driver.nwchem2yaml import extract_fields
from qiskit_nwchem_driver import nwchem_driver
import yaml
from pyscf import fci
import matplotlib.pyplot as plt
import numpy as np


fcisolver = fci.direct_uhf.FCISolver()


covos = [1, 4, 8, 12, 18]
data_covos = {}
for covo in covos:
    print('{} COVOs'.format(covo))
    data_dir_nwchem = os.path.join("..","data","PW_LiH_data", "3x3_aperiodic", 'NWChem','{}covo'.format(covo))
    data_dir_yaml = os.path.join("..","data","PW_LiH_data", "3x3_aperiodic", 'NWChem', '{}covo_yaml'.format(covo))
    if not os.path.exists(data_dir_yaml):
        os.makedirs(os.path.join("..","data","PW_LiH_data", "3x3_aperiodic", 'NWChem', '{}covo_yaml'.format(covo)))
    data_files = os.listdir(data_dir_nwchem)
    bond_distances = []
    total_energies = []

    for data_file in data_files:
        if data_file.find('out') == -1:
            continue
        temp = data_file.split('-')
        temp1 = temp[1].split('.')
        bond_distance = float(temp1[0] +'.'+ temp1[1])
        bond_distances.append(bond_distance)
        name = temp[0] + '-' + str(bond_distance)
        print("========= Bond distance: {} =========".format(bond_distance))
        data_file_yaml = os.path.join(data_dir_yaml, "{}.yaml".format(name))
        data = extract_fields(os.path.join(data_dir_nwchem,data_file))
        with open(data_file_yaml, 'w') as f:
            f.write(yaml.dump(data, default_flow_style=False)) 
        
        driver = nwchem_driver.NWchem_Driver(data_file_yaml)
       
        n_electrons, n_spatial_orbitals, nuclear_repulsion_energy, h1, h2 = driver.load_from_yaml(data_file_yaml, include_spin=False)
     
        energy, coefficients = fcisolver.kernel(
            h1e=(h1, h1),  
            eri=(h2, h2, h2),  
            norb=n_spatial_orbitals,
            nelec=(1, 1),
            nroots=1
        )
        
        print(energy)
        
        total_energies.append(energy + nuclear_repulsion_energy)
    print(total_energies)

    total_energies = np.array(total_energies)
    bond_distances = np.array(bond_distances)

    sorted_indices = np.argsort(bond_distances)
    bond_distances = bond_distances[sorted_indices]
    total_energies = total_energies[sorted_indices]
    data_covos[covo] = np.array([bond_distances, total_energies])



energies_true_pvdz_LiH = [result['energy'] for result in loaded_results_true_pvdz_LiH]   
energies_false_pvdz_LiH = [result['energy'] for result in loaded_results_false_pvdz_LiH]   
energies_true_pvtz_LiH = [result['energy'] for result in loaded_results_true_pvtz_LiH]   
energies_false_pvtz_LiH = [result['energy'] for result in loaded_results_false_pvtz_LiH]   

for covo in data_covos:
    plt.plot(data_covos[covo][0], data_covos[covo][1], 'o-', label=f'FCI {covo} COVO')
plt.plot(data_covos[covo][0],energies_true_pvdz_LiH, 'x-',label='oo-cc-pvdz with 10 orbitals')
plt.plot(data_covos[covo][0],energies_false_pvdz_LiH, label='cc-pvdz with 10 orbitals')
plt.plot(data_covos[covo][0],energies_true_pvtz_LiH, 'x-', label='oo-cc-pvtz with 10 orbitals')
plt.plot(data_covos[covo][0],energies_false_pvtz_LiH, label='cc-pvtz with 10 orbitals')
plt.legend(loc=1)
plt.xlabel('Bond distance (Å)')
plt.ylabel('Energy (Hartree)')
plt.title('LiH Energy w.r.t bond distance')
#plt.ylim([-1.18,-0.8])
plt.savefig('LiH_COVO.png')
plt.show()